# Log

In [2]:
import os, sys
from loguru import logger

LOG_ROOT = os.path.abspath("./")
LOG_FILE = LOG_ROOT + "/logs/mimic-cxr.log"

# Remove all handlers and reset stderr
logger.remove(handler_id=None)
logger.add(
    LOG_FILE,
    level="TRACE",
    mode="w",
    backtrace=False,
    diagnose=True,
    colorize=False,
    format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}",
)
logger.info("\r\n" + ">" * 29 + "\r\n" + ">>> New execution started >>>" + "\r\n" + ">" * 29)
# To filter log level: TRACE=5, DEBUG=10, INFO=20, SUCCESS=25, WARNING=30, ERROR=40, CRITICAL=50
logger.add(sys.stdout, level="INFO", filter=lambda record: record["level"].no < 40, colorize=True)
logger.add(sys.stderr, level="ERROR", backtrace=False, diagnose=True, colorize=True)

3

# Read From Database

In [3]:
import os
import pymysql
import pandas as pd

# DB connect
DB = pymysql.connect(host="127.0.0.1", port=3306, user="root", password="1121", db="radiology_datasets", charset="utf8")
CURSOR = DB.cursor(cursor=pymysql.cursors.DictCursor)

sql_select = "SELECT * FROM radiology_datasets.`mimic-cxr_manual`;"

CURSOR.execute(sql_select)
df = pd.DataFrame(CURSOR.fetchall())
print(df)

CURSOR.close()
DB.close()

              pid        sid  \
0       p10000032  s50414267   
1       p10000032  s53189527   
2       p10000032  s53911762   
3       p10000032  s56699142   
4       p10000764  s57375967   
...           ...        ...   
227830  p19999442  s58708861   
227831  p19999733  s57132437   
227832  p19999987  s55368167   
227833  p19999987  s58621812   
227834  p19999987  s58971208   

                                                 findings  \
0       There is no focal consolidation, pleural effus...   
1       The cardiac, mediastinal and hilar contours ar...   
2       Single frontal view of the chest provided. \n ...   
3       The lungs are clear of focal consolidation, pl...   
4       PA and lateral views of the chest provided.   ...   
...                                                   ...   
227830  ET tube ends 4.7 cm above the carina.  NG tube...   
227831  The lungs are clear, and the cardiomediastinal...   
227832  There has been interval extubation and improve...   
22783

# Utils


## Remove heading labels
@[...]

In [4]:
import re

def removeHeadingLabels(content):
    reStr = r"@\[.*?\]"
    content = re.sub(reStr,"",content)
    return content.strip()

def find_headingLabels(content):
    reStr = r"@\[.*?\]"
    m = re.findall(reStr,content)
    return m

def headingLabelsCounter(content):
    return len(find_headingLabels(content))

# Data checking

## Sections counting
Results: 

Regarding the findings, impression, PFI, and FAI sections, 217,240 reports have at least one of the findings or impression section. As for the rest, 10,588 reports have the FAI section. Only 7 reports have no content. 

227,835 reports in total.

In [5]:
print(f"Reports: {df.shape[0]}")

print("Reports that ")
df_F = df.query("findings != ''")
print(f"    contain the FINDINGS section: {df_F.shape[0]}")

df_I = df.query("impression != ''")
print(f"    contain the IMPRESSION section: {df_I.shape[0]}")

df_PFI = df.query("provisional_findings_impression != ''")
print(f"    contain the PFI section: {df_PFI.shape[0]}")

df_FAI = df.query("findings_and_impression != ''")
print(f"    contain the FAI section: {df_FAI.shape[0]}")

df_F_or_I = df.query("(findings != '' | impression != '')")
print(f"    contain at least one of the FINDINGS or IMPRESSION section: {df_F_or_I.shape[0]}")

df_F_I = df.query("(findings != '' & impression != '')")
print(f"    contain both the FINDINGS and IMPRESSION sections: {df_F_I.shape[0]}")

df_noF_noI_PFI_or_FAI = df.query("(findings == '' & impression == '') & (provisional_findings_impression != '' | findings_and_impression != '')")
print(f"    have at least one of the PFI or FAI section: {df_noF_noI_PFI_or_FAI.shape[0]}")

df_noF_noI_PFI_FAI = df.query("(findings == '' & impression == '') & (provisional_findings_impression != '' & findings_and_impression != '')")
print(f"    have both the PFI and FAI sections: {df_noF_noI_PFI_FAI.shape[0]}")


print("\nReports that do not contain the provisional_findings_impression or findings_and_impression sections, but ")
df_noPFI_noFAI_F_or_I = df.query("(findings != '' | impression != '') & (provisional_findings_impression == '' & findings_and_impression == '')")
print(f"    contain at least one of the FINDINGS or IMPRESSION section: {df_noPFI_noFAI_F_or_I.shape[0]}")

df_noPFI_noFAI_F_I = df.query("(findings != '' & impression != '') & (provisional_findings_impression == '' & findings_and_impression == '')")
print(f"    contain both the FINDINGS and IMPRESSION sections: {df_noPFI_noFAI_F_I.shape[0]}")

df_noPFI_noFAI_F_only = df.query("findings != '' & impression == '' & provisional_findings_impression == '' & findings_and_impression == ''")
print(f"    only contain the FINDINGS section: {df_noPFI_noFAI_F_only.shape[0]}")

df_noPFI_noFAI_I_only = df.query("findings == '' & impression != '' & provisional_findings_impression == '' & findings_and_impression == ''")
print(f"    only contain the IMPRESSION section: {df_noPFI_noFAI_I_only.shape[0]}")


print("\nReports that do not contain the FINDINGS or IMPRESSION section, but")
df_noF_noI_PFI_or_FAI = df.query("(findings == '' & impression == '') & (provisional_findings_impression != '' | findings_and_impression != '')")
print(f"    have at least one of the PFI or FAI section: {df_noF_noI_PFI_or_FAI.shape[0]}")

df_noF_noI_PFI_FAI = df.query("(findings == '' & impression == '') & (provisional_findings_impression != '' & findings_and_impression != '')")
print(f"    have both the PFI and FAI sections: {df_noF_noI_PFI_FAI.shape[0]}")

df_noF_noI_PFI_only = df.query("findings == '' & impression == '' & provisional_findings_impression != '' & findings_and_impression == ''")
print(f"    only have the PFI section: {df_noF_noI_PFI_only.shape[0]}")

df_noF_noI_FAI_only = df.query("findings == '' & impression == '' & provisional_findings_impression == '' & findings_and_impression != ''")
print(f"    only have the FAI section: {df_noF_noI_FAI_only.shape[0]}")


df_empty = df.query("findings == '' & impression == '' & provisional_findings_impression == '' & findings_and_impression == ''")
print(f"Empty reports: {df_empty.shape[0]}")

Reports: 227835
Reports that 
    contain the FINDINGS section: 156260
    contain the IMPRESSION section: 189493
    contain the PFI section: 200
    contain the FAI section: 10591
    contain at least one of the FINDINGS or IMPRESSION section: 217240
    contain both the FINDINGS and IMPRESSION sections: 128513
    have at least one of the PFI or FAI section: 10588
    have both the PFI and FAI sections: 2

Reports that do not contain the provisional_findings_impression or findings_and_impression sections, but 
    contain at least one of the FINDINGS or IMPRESSION section: 217039
    contain both the FINDINGS and IMPRESSION sections: 128317
    only contain the FINDINGS section: 27745
    only contain the IMPRESSION section: 60977

Reports that do not contain the FINDINGS or IMPRESSION section, but
    have at least one of the PFI or FAI section: 10588
    have both the PFI and FAI sections: 2
    only have the PFI section: 0
    only have the FAI section: 10586
Empty reports: 7


## Having multiple labels in one section

Result: 

No more than 1 heading per section. The four heading labels in the impression section are "@[manual]".

In [6]:
df_num = df.loc[:,'findings':'findings_and_impression'].applymap(lambda ele: headingLabelsCounter(str(ele)))
f_gt_1 = df_num.loc[lambda _df: _df['findings'] > 0].shape
i_gt_1 = df_num.loc[lambda _df: _df['impression'] > 0].shape
pfi_gt_1 = df_num.loc[lambda _df: _df['provisional_findings_impression'] > 1].shape
fai_gt_1 = df_num.loc[lambda _df: _df['findings_and_impression'] > 1].shape

print("Reports that have more that one heading labels in the same section:")
print(f"    findings, label_num > 0: {f_gt_1[0]}")
print(f"    impression, label_num > 0: {i_gt_1[0]}")
print(f"    provisional_findings_impression, label_num > 1: {pfi_gt_1[0]}")
print(f"    findings_and_impression, label_num > 1: {fai_gt_1[0]}")

Reports that have more that one heading labels in the same section:
    findings, label_num > 0: 0
    impression, label_num > 0: 4
    provisional_findings_impression, label_num > 1: 0
    findings_and_impression, label_num > 1: 0


# To JSON/XML file

In [ ]:
df_no_labels = df.loc[:,'pid':'findings_and_impression'].applymap(lambda ele: removeHeadingLabels(str(ele)))

In [9]:
df_no_labels.to_json(os.path.join(os.path.abspath("./"),"mimic_cxr_reports_core.json"),orient="records",lines=True)
# df_no_labels.to_xml(os.path.join(os.path.abspath("./"),"mimic_cxr_reports_core.xml"),)

# Visualization & Statistics

## Entities

Load entities

In [38]:
ent_files = ['findings.txt','impression.txt','FAI.txt','PFI.txt']
ent_list = []
for ent_file in ent_files:
    with open('/Users/liao/myProjects/VSCode_workspace/structured_reporting/statistic/'+ent_file,'r') as f:
        ent_list.extend(f.readlines())
print(f"Input rows: {len(ent_list)}")      
def filterFunc(x):
    try:
        m = re.match(r"s\d+:",x)
        m.group()
        return True
    except AttributeError:
        return False
ent_list = list(filter(filterFunc,ent_list))
print(f"After filter: {len(ent_list)}")


Input rows: 3738313
After filter: 3738246


Process

In [37]:
from collections import Counter

entCounter = Counter([row.split(":")[1].strip() for row in ent_list])
print(len(entCounter))
for i in entCounter.most_common(5):
    print(i)

85180
('pneumothorax', 112997)
('pleural effusion', 91543)
('lungs', 52227)
('acute', 46120)
('unchanged', 43721)


Output CSV

In [36]:
import csv

data_list = [['Entity','Counting']]
for ent,num in entCounter.most_common():
    data_list.append([ent,num])

with open("ent.csv",mode="w",newline="") as f:
    csv.writer(f).writerows(data_list)